In [1]:
from pymoo.problems import get_problem
from tqdm import tqdm
import pandas as pd
from utils import *
from pymoo.vendor.vendor_coco import COCOProblem
import random
import os

In [2]:
dim=10
n_samples=100
n_evals = 250

In [3]:
dfs = []

for coco_problem in range(1, 9):
    for coco_instance in range(1, 12):
        problem = COCOProblem(f"bbob-f{coco_problem}-{coco_instance}", n_var=dim)
        data = run_algorithms(problem, n_runs=5, algorithms=['GA', 'CMAES'], n_eval=n_evals)
        df = pd.DataFrame(data)
        df['problem_class'] = coco_problem
        df['problem_instance'] = coco_instance
        df['optimum'] = problem.evaluate(problem.pareto_set())[0,0]
        #sampling = LHS()
        #X = sampling(problem, dim*n_samples).get("X")
        #y = problem.evaluate(X).flatten()
        dfs.append(df)
df = pd.concat(dfs)      

Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 evals
Using 2500 eva

In [4]:
df

,GA,CMAES,algorithm_run,problem_class,problem_instance,optimum
0,79.481671,79.480000,0,1,1,79.48
1,79.482633,79.480000,1,1,1,79.48
2,79.481640,79.480000,2,1,1,79.48
3,79.481200,79.480000,3,1,1,79.48
4,79.481557,79.480000,4,1,1,79.48
...,...,...,...,...,...,...
0,598.172999,588.262154,0,8,11,582.15
1,585.209275,585.555184,1,8,11,582.15
2,588.947297,586.150352,2,8,11,582.15
3,590.851358,586.502661,3,8,11,582.15


In [5]:
df['GA_perf'] = np.log(df['GA']-df['optimum'])
df['CMAES_perf'] = np.log(df['CMAES']-df['optimum'])

In [6]:
gdf = df.groupby(['problem_class', 'problem_instance']).mean().reset_index()
gdf

,problem_class,problem_instance,GA,CMAES,algorithm_run,optimum,GA_perf,CMAES_perf
0,1,1,79.481740,79.480000,2.0,79.48,-6.387495,-21.589692
1,1,2,394.481699,394.480000,2.0,394.48,-6.530845,-18.760341
2,1,3,-247.107938,-247.110000,2.0,-247.11,-6.283424,-19.554033
3,1,4,-152.038739,-152.040000,2.0,-152.04,-6.828294,-20.796031
4,1,5,-25.248925,-25.250000,2.0,-25.25,-6.887777,-20.981779
...,...,...,...,...,...,...,...,...
83,8,7,76.345544,70.970127,2.0,66.03,2.152407,1.575792
84,8,8,218.563567,176.199776,2.0,172.01,2.792690,1.402080
85,8,9,16.656184,-21.745700,2.0,-27.25,3.151029,1.664594
86,8,10,-94.775529,-147.224058,2.0,-152.18,3.238344,1.568975


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [8]:
for problem_instance in gdf.problem_instance.unique():
    train = gdf.query(f'problem_instance!={problem_instance}')
    test = gdf.query(f'problem_instance=={problem_instance}')
    
    for feature_columns in [['problem_class'], ['problem_class', 'GA_perf']]:
        X_train = train[feature_columns]
        X_test = test[feature_columns]
        y_train = train['GA_perf']
        y_test = test['GA_perf']

        rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_regressor.fit(X_train, y_train)
        pred = rf_regressor.predict(X_test)
        error = mean_absolute_error(y_test, pred)
        print(feature_columns, error)

['problem_class'] 0.5568946126462964
['problem_class', 'GA_perf'] 0.036226152505212744
['problem_class'] 0.2635743415684513
['problem_class', 'GA_perf'] 0.027204533448996704
['problem_class'] 0.34929726347669265
['problem_class', 'GA_perf'] 0.03717184897760077
['problem_class'] 0.4020034640227739
['problem_class', 'GA_perf'] 0.02348016235354062
['problem_class'] 0.46150440022284933
['problem_class', 'GA_perf'] 0.03929524851414727
['problem_class'] 0.4253836194518347
['problem_class', 'GA_perf'] 0.11569466942306808
['problem_class'] 0.4590980979518652
['problem_class', 'GA_perf'] 0.033263638342876246
['problem_class'] 0.36511880375269085
['problem_class', 'GA_perf'] 0.05456320064349601
['problem_class'] 0.5001260867298124
['problem_class', 'GA_perf'] 0.09758258336415765
['problem_class'] 0.488933425587895
['problem_class', 'GA_perf'] 0.0691470389318454
['problem_class'] 0.4072207903381464
['problem_class', 'GA_perf'] 0.03628615856398523


In [9]:
for problem_class in gdf.problem_class.unique():
    train = gdf.query(f'problem_class!={problem_class}')
    test = gdf.query(f'problem_class=={problem_class}')
    
    for feature_columns in [['problem_class'], ['problem_class', 'CMAES_perf']]:
        X_train = train[feature_columns]
        X_test = test[feature_columns]
        y_train = train['GA_perf']
        y_test = test['GA_perf']

        rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_regressor.fit(X_train, y_train)
        pred = rf_regressor.predict(X_test)
        error = mean_absolute_error(y_test, pred)
        print(feature_columns, error)

['problem_class'] 9.61414797245459
['problem_class', 'CMAES_perf'] 2.2576240390548574
['problem_class'] 9.609998448407183
['problem_class', 'CMAES_perf'] 2.0167070773908744
['problem_class'] 2.2114199172338753
['problem_class', 'CMAES_perf'] 1.8462233819567806
['problem_class'] 0.6399012771934435
['problem_class', 'CMAES_perf'] 0.9951147184681081
['problem_class'] 10.447121611111982
['problem_class', 'CMAES_perf'] 9.252235673961373
['problem_class'] 10.08378105687952
['problem_class', 'CMAES_perf'] 0.9951960925851114
['problem_class'] 1.03856411104531
['problem_class', 'CMAES_perf'] 4.642115005410763
['problem_class'] 0.6686221302357858
['problem_class', 'CMAES_perf'] 0.755956793016976


In [10]:
train

,problem_class,problem_instance,GA,CMAES,algorithm_run,optimum,GA_perf,CMAES_perf
0,1,1,79.481740,79.480000,2.0,79.48,-6.387495,-21.589692
1,1,2,394.481699,394.480000,2.0,394.48,-6.530845,-18.760341
2,1,3,-247.107938,-247.110000,2.0,-247.11,-6.283424,-19.554033
3,1,4,-152.038739,-152.040000,2.0,-152.04,-6.828294,-20.796031
4,1,5,-25.248925,-25.250000,2.0,-25.25,-6.887777,-20.981779
...,...,...,...,...,...,...,...,...
72,7,7,-315.613699,-321.016965,2.0,-321.72,1.742649,-0.581859
73,7,8,-90.692987,-103.311961,2.0,-104.20,2.472520,-5.466185
74,7,9,73.381865,61.472001,2.0,61.47,2.320062,-18.784486
75,7,10,-14.880833,-26.009011,2.0,-27.54,2.370469,-8.639497


In [11]:
#!pip install Boruta

In [12]:
#import numpy as np
#import pandas as pd
#from sklearn.datasets import make_classification
#from sklearn.ensemble import RandomForestClassifier
#from boruta import BorutaPy
#
#np.int = np.int32
#np.float = np.float64
#np.bool = np.bool_
#
## Generate some example data
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=2, random_state=1)
#
#rf = RandomForestClassifier(n_jobs=-1)
#
## define Boruta feature selection method
#feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
#
## find all relevant features - 5 features should be selected
#feat_selector.fit(X, y)
#
## check selected features - first 5 features are selected
#feat_selector.support_
#
## check ranking of features
#feat_selector.ranking_
#
## call transform() on X to filter it down to selected features
#X_filtered = feat_selector.transform(X)

In [13]:
#feat_selector.ranking_